# Simple Wing with Unsteady Aerodynamics
After the steady aerodynamic case using the Vortec Lattice Method (VLM), this tutorial demonstrates on how to use the unsteady Doublet Lattice Method (DLM) at the example of the same, simple wing. Assuming that you already did the steady case, we will shorten/skip some of the explanations.

It is also assumed that you sucessfully installed PanelAero as a Python package as described in the [README](https://github.com/DLR-AE/PanelAero?tab=readme-ov-file#installation--use).

In addition to the DLM inself, this tutorial also uses numpy and some helper functions provided along with the tutorials.

In [ ]:
import numpy as np

from panelaero import DLM

from tutorials.helper_functions import build_aeromodel, plotting

## Set-up of Geometry
As before, we parse the aerodynamic grid of our simple, rectangular wing defined by the CAERO1 card.

In [ ]:
model = build_aeromodel.AeroModel('./simplewing.CAERO1')
model.build_aerogrid()
plots = plotting.DetailedPlots(model)
plots.plot_aerogrid(embed_in_notebook=True)

## Calculate the AIC Matrix
The Doublet Lattice Method (DLM) uses the same geoemtrical description of the lifting surface with aeroynamic panels as the VLM. However, the underlying modeling uses doublets insteady of vortices and works in the frequency domain. 

Like the VLM, the DLM provided in this software calculates a matrix of so-called aerodynamic influence coefficients (AIC), which depends on the geometry, the Mach number and the reduced frequency, defined by

$$k = \frac{ \omega }{V}$$

Note that the "Nastran definition" of the reduced frequency adds $c_\textup{ref}/2$, leading to 

$$k = {c_{ref} \over 2V} \cdot \omega$$

As before, the $\mathbf{AIC}$ matrix then relates the induced downwash $\mathbf{w_j}$ on each aerodynamic panel to complex pressure coefficients $\mathbf{\Delta c_p}$.

$$\mathbf{\Delta c_p} = \mathbf{AIC}(Ma, k) \cdot \mathbf{w_j}$$

For more information on the theoretical background of the DLM, please consult Section 2.3 in the Technical Report DLR-IB-AE-GO-2020-137.

The $\mathbf{AIC}$ matrix (called Qjj in the following) is obtained in the following way:

In [ ]:
Qjj = DLM.calc_Qjj(model.aerogrid, Ma=0.0, k=0.1)

Next, let's assume the same onflow condition with Vtas = 25.0 m/s and an induced downwash of wj = 2.18 m/s on every panels, which corresponds to an angle of attack = 5.0 deg. Remember that the downwash is scaled with Vtas.

In [ ]:
Vtas = 25.0
wj = np.ones(model.aerogrid['n']) * 2.18 / Vtas

Following the formula given above, multiplication of the downwash vector $\mathbf{w_j}$ with the $\mathbf{AIC}$ matrix yields the pressure coefficient distribution $\mathbf{\Delta c_p}$.

In [ ]:
cp = Qjj.dot(wj)

## Vizualisation of Results
The pressure coefficient distribution may be visualized on the geometry using the helper function provided along with this tutorial. Becasue we selected $k=0.1$ above, the pressure coefficients will be complex, so we vizualize the real and the imaginary part seperately. Compared to the steady aerodynamic approach using the VLM, the complex pressure coefficients obtained from the DLM change the magnitude and add a phase shift depending on the prescribed reduced frequency $k$.

In [ ]:
plots.plot_aerogrid(cp.real, embed_in_notebook=True)

In [ ]:
plots.plot_aerogrid(cp.imag, embed_in_notebook=True)

## Advanced Use
For most aeroelastic applications, typically the $\mathbf{AIC}$ matrices are needed for different Mach numbers and for multiple reduced frequencies $k$. The function DLM.calc_Qjjs() accepts a lists as input and returns a four-dimensional array with the shape (n_mach, n_k, n_panels, n_panels] as shown in the example below.

In [ ]:
Qjjs = DLM.calc_Qjjs(model.aerogrid, Ma=[0.0, 0.5], k=[0.01, 0.1, 0.3, 0.5])
Qjjs.shape